# Краткая инструкция

К системе подключен GRASS и полноценный терминал.

Система в целом "вандалоустойчивая" так что не бойтесь экспериментировать. При соблюдении следующих рекомендаций, проблем не будет:
 
 * У вас есть доступ на запись в PERMANENT, в котором хранятся основные данные. Но экспериментальную работу желательно вести работу в отдельном MAPSET, чтобы ненароком не испортить исходные данные (в GRASS можно читать данные из любых MAPSET, а удалять/записывать в текущий). Как подключиться к GRASS и создать/перейти в отдельный MAPSET сказано ниже.
 * Система работает из-под одного пользователя (klsvd), весь код хранится в репозитории. Понимаю, что не очень удобно работать из-под одного пользователя, но пока так. При необходимости закоммитить результаты работы и отправить их в репу -- обращайтесь ко мне (КД). Чтобы не испортить работу других, желательно создать для тестов отдельный блокнот-песочницу и отлаживаться в нем.
 
Это именно рекомендации, а не жесткие требования.

Ниже идут подробности процесса. В первый раз лучше прочитать все подряд, в дальнейшем можно пользоваться шпаргалкой:

 * [Как подключаться к GRASS](#connect_grass).
 * [Как создать MAPSET](#create_mapset).
 * [Полный цикл -- создание области, обработка, просмотр результатов](#all_steps).
 * [Как пользоваться WMS](#wms).

## Python

В настоящий момент к блокнотам подключен только Python версии 2.7. Есть возможность подключить другие языки, в том числе R. Если нужно - сообщайте. В блокнотах дается доступ к "обычному" Python, т.е. все, что можно сделать с ним в терминале, можно сделать и в блокноте. Для этого нужно набрать требуемый код в ячейке и нажать кнопку 'Run cell' (или набрать на клавиатуре сочетание клавиш Ctrl-Enter).

In [1]:
import gdal

print 'Версия gdal:', gdal.VersionInfo()

x = 10
print x * 200

Версия gdal: 1110200
2000


## Подключение к GRASS<a id='connect_grass'></a>

Для удобства работы с GRASS был написан объект, хранящийся в модуле grasslib. Поэтому вначале его следует импортировать. Для запуска GRASS нужно знать пути к исполняемому коду и каталогу с данными. Для удобства эти пути прописаны в одном месте (в случае переезда нужно будет менять только один файл, а не лазить по всем блокнотам), поэтому модуль, хранящий пути, также следует импортировать. Помимо путей в нем лежат несколько других полезных функций (модуль называется utilites, можете открыть и посмотреть, что там внутри).

Импортируем необходимые модули:

In [2]:
import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_location_path,
)

Создадим объект - обертку над GRASS, для этого нужно передать ему пути подключения к исполняемому коду, каталогу с данными GRASS, название LOCATION и (опционально) MAPSET. Если MAPSET не указан, то по умолчанию подключение производится к PERMANENT.

In [3]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name(),
            mapset='PERMANENT'
)

Начиная с этого момента можно использовать обертку для исполнение команд в GRASS (через `grs.grass.*`). Для работы доступно все, что доступно в GRASS из Python, т.е. практически без ограничений. Наиболее важные функции: run_command(), read_command(), write_command(), их хватает на 99% обычной работы (подробности можно посмотреть в [презентации с gisconf](http://kolesovdmitry.github.io/gisconf-grass-scripting/slides) и в документации к GRASS). Например:

In [4]:
grs.grass.run_command('g.list', type='rast')
x = grs.grass.read_command('g.list', type='vect')
print x

train15_16



## Создание MAPSET<a id='create_mapset'></a>

Для того, чтобы не мусорить в PERMANENT, эксперименты рекомендуется проводить в отдельных MAPSETах. Ниже показан пример создания MAPSET под названием test и переключение в него при помощи команды `g.mapset`. Флаг `-c` позволяет создать новый MAPSET, если этот флаг не указан, то будет произведено переключение в существующий (если его существует, будет выведена ошибка). 

In [5]:
# Выведем список всех известных системе MAPSET
print 'Существующие MAPSET:', grs.grass.read_command('g.mapset', flags='l')

# Выведем текущий MAPSET
print 'Текущий MAPSET:', grs.grass.read_command('g.mapset', flags='p')

grs.grass.run_command('g.mapset', mapset='test', flags='c')

print 'Текущий MAPSET:', grs.grass.read_command('g.mapset', flags='p')

Существующие MAPSET: PERMANENT exploration test 

Текущий MAPSET: PERMANENT

Текущий MAPSET: test



Так же можно подключиться к нужному MAPSET в момент инициализации GRASS (см. раздел о подключении к GRASS).

## Обработка данных и просмотр результатов<a id='all_steps'></a>

Покажем этот процесс на примере. Предположим, нам нужно рассчитать NDVI по одной из сцен и посмотреть, что получилось. Для этого создадим в MAPSET с названием `test` требуемый растр и подключим его через WMS в ГИС. 

Последовательность действий:
1. Переключимся в `test` (если мы еще не там).
2. Выберем сцену.
3. Рассчитаем NDVI.
4. Подключаемся к WMS.

### Шаг 1: переключение в MAPSET.

In [6]:
mapset = grs.grass.read_command('g.mapset', mapset='test', flags='p')
print mapset

test



### Шаг 2: список растров (групп растров).

Выведем список известных сцен (воспользуемся тем, что они сгруппированы, см. блокнот 000_Initialization).

In [7]:
groups = grs.grass.read_command('g.list', type='group')
print groups

LC81120272015157LGN00
LC81120272015189LGN00
LC81120272015333LGN00
LC81120272015365LGN00
LC81120272016032LGN01
LC81120282015141LGN00
LC81120282015189LGN00
LC81120282015237LGN00
LC81120282015317LGN00
LC81120282015365LGN00
LC81120282016016LGN00
LC81120282016032LGN01
toar_LC81120272015157LGN00
toar_LC81120272015189LGN00
toar_LC81120272015333LGN00
toar_LC81120272015365LGN00
toar_LC81120272016032LGN01
toar_LC81120282015141LGN00
toar_LC81120282015189LGN00
toar_LC81120282015237LGN00
toar_LC81120282015317LGN00
toar_LC81120282015365LGN00
toar_LC81120282016016LGN00
toar_LC81120282016032LGN01



Выберем одну из этих сцен, например, LC81120282016032LGN01. По ней уже рассчитаны ToA Reflectance, поэтому можно считать NDVI. (Напомним, что основные данные хранятся в PERMANENT). Установим регион по одному из ратров сцены и запустим расчеты через растровый калькулятор.

### Шаг 3: обработка.

In [8]:
grs.grass.run_command('g.region', raster="toar_LC81120282016032LGN01_B5@PERMANENT")

expr = "ndvi = (%s - %s)/(%s + %s)" % \
    ('toar_LC81120282016032LGN01_B5@PERMANENT', 'toar_LC81120282016032LGN01_B4@PERMANENT',
     'toar_LC81120282016032LGN01_B5@PERMANENT', 'toar_LC81120282016032LGN01_B4@PERMANENT')


grs.grass.run_command('r.mapcalc', expression=expr, overwrite=True)

print grs.grass.read_command('r.info', map='ndvi')

 +----------------------------------------------------------------------------+
 | Map:      ndvi                           Date: Sat Apr 16 10:59:30 2016    |
 | Mapset:   test                           Login of Creator: klsvd           |
 | Location: LANDSAT                                                          |
 | DataBase: /home/klsvd/GRASSDATA                                            |
 | Title:     ( ndvi )                                                        |
 | Timestamp: none                                                            |
 |----------------------------------------------------------------------------|
 |                                                                            |
 |   Type of Map:  raster               Number of Categories: 0               |
 |   Data Type:    DCELL                                                      |
 |   Rows:         7911                                                       |
 |   Columns:      7781                 

Как видим, растр создан, для удобства назначим ему цвета по стандартной схеме цветов ndvi:

In [9]:
grs.grass.run_command('r.colors', map='ndvi', color='ndvi')

0

### Шаг 4: WMS.<a id='wms'></a>

WMS сервер доступен по адресу: `http://[ip-адрес]/wms/Название-MAPSET/`. По этому адресу можно подключиться в любой ГИС и просмотреть содержимое запрошенного MAPSET и PERMANENT (данные, хранящиеся в PERMANENT, доступны для просмотра из любого MAPSET).

Например, полученный нами растр NDVI будет доступен по адресу: `http://[ip-адрес]/wms/test/`.